# **Lumping Circuits for State preparation**

In [1]:
import sys; sys.path.insert(0, "../..") # clue is here
from clue.qiskit import *
from numpy import array, ndarray, matmul, cdouble, sqrt, arccos, cos, sin
from numpy.random import random
from qiskit import execute, QuantumCircuit, Aer

The quantum state preparation is an important part of the quantum circuits that essentially transforms the quantum state $\ket 0$ to any specific quantum state $\ket{\psi}$. We could try to apply lumping for this type of circuits in two different ways:

* Set as input the state $\ket 0$. This will produce a vector space related with the forward invariance of the zero state and producing an easy way to apply the circuit to the zero state.
* Set as input the state $\ket \psi$. This will produce a lumping for the goal state where we could ehck if the zero state is there and if it produces the desired state.

The first step towards applying this systematically is to grasp control on the unitary matrix defined by the circuits that perform this state preparation. Then we need to create the corresponding CLUE systems and produce the final lumpings for all the inputs.

## 1. Getting matrix for Circuit for State Preparation

In [15]:
def is_vector(v: ndarray):
    return isinstance(v, ndarray) and len(v.shape) == 1

def unitary_preparation(v: ndarray, *, _tol = 1e-10):
    if not is_vector(v): raise TypeError(f"Required a vector for state preparation")
    nbits = int(log2(v.shape[0]))
    if 2**nbits != v.shape[0]: raise ValueError("The vector should be of size 2^n")
    
    if abs(matmul(v,v) - 1) > _tol: # making `v` of norm 1
        v = v / sqrt(matmul(v,v))
        
    circuit = QuantumCircuit(nbits)
    circuit.initialize(v)
    job = execute(circuit, DS_QuantumCircuit.BACKEND, shots=8192)
    return v, job.result().get_unitary(circuit)

In [16]:
unitary_preparation(random(2))

(array([0.64369642, 0.76528094]),
 Operator([[ 0.64369642+0.j, -0.76528094+0.j],
           [ 0.76528094+0.j,  0.64369642+0.j]],
          input_dims=(2,), output_dims=(2,)))

## 2. Creating the system from the circuit and the observable

In [19]:
state, circuit = unitary_preparation(random(128))
system = DS_QuantumCircuit(circuit, name="initializer")
zero = SparsePolynomial.from_string(system.variables[0], system.variables, system.field)
goal = SparsePolynomial.from_vector(state, system.variables, system.field)

In [23]:
# lumping from the goal state
lumped_goal = system.lumping([goal], print_reduction=False, print_system=False); lumped_goal

Lumped system [128 -> 128] (initializer) [LDESystem -- 128 -- SparsePolynomial]

In [24]:
# lumping from the zero state
lumped_zero = system.lumping([zero], print_reduction=False, print_system=False); lumped_zero

Lumped system [128 -> 128] (initializer) [LDESystem -- 128 -- SparsePolynomial]

We can see that for a random state we do not reduce in any of the cases (which makes complete sense).

## 3. Trying specific states

From a random state is normal we can not reduce anything. But what could happend in specific states. For example, GHZ-state is easyly defined as a vector and we know from our experiments with the benchmarks that this can be reduced to a 4 qubit circuit. Can we repeat this result from this initializer?

In [48]:
nbits = 4
state, circuit = unitary_preparation(array([1] + (2**nbits - 2)*[0] + [1]))
system = DS_QuantumCircuit(circuit, name="initializer")
zero = SparsePolynomial.from_string(system.variables[0], system.variables, system.field)
goal = SparsePolynomial.from_vector(state, system.variables, system.field)

In [27]:
# lumping from the goal state
lumped_goal = system.lumping([goal], print_reduction=False, print_system=False); lumped_goal

Lumped system [128 -> 128] (initializer) [LDESystem -- 128 -- SparsePolynomial]

In [28]:
# lumping from the zero state
lumped_zero = system.lumping([zero], print_reduction=False, print_system=False); lumped_zero

Lumped system [128 -> 128] (initializer) [LDESystem -- 128 -- SparsePolynomial]

It is curious that we do not see the same reduction. Let us compare the two circuits, namely, let us build the matrix for the initializer and from the ghz file and see what are the differences:

In [49]:
system_ghz = DS_QuantumCircuit.from_qasm_file(f"./circuits/ghz_indep_qiskit_{nbits}.qasm")
A1 = system.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble); A2 = system_ghz.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble)

In [55]:
((A1-A2).round(10) == 0).all()

False